# Fundamental 21. 딥러닝 레이어의 이해 - Embedding, Recurrent

## Embedding 레이어
### 원-핫 인코딩  One-Hot Encoding

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


생성된 원-핫 벡터를 Linear 레이어에 넣어보자.

In [2]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[ 0.14386714 -0.28959525]
 [ 0.01624763 -0.47682944]
 [-0.32366136 -0.49629986]
 [ 0.10942185  0.19583851]
 [ 0.2834351   0.5339401 ]
 [ 0.18905717 -0.32596153]
 [-0.03354037 -0.70990115]
 [-0.487678    0.47108424]]

One-Hot Linear Result
[[ 0.14386714 -0.28959525]
 [ 0.14386714 -0.28959525]
 [ 0.14386714 -0.28959525]
 [ 0.14386714 -0.28959525]
 [ 0.01624763 -0.47682944]
 [-0.32366136 -0.49629986]
 [ 0.10942185  0.19583851]
 [ 0.2834351   0.5339401 ]
 [ 0.2834351   0.5339401 ]
 [ 0.2834351   0.5339401 ]]


원-핫 벡터에 Linear 레이어를 적용하니 Linear 레이어의 Weight에서 단어 인덱스 배열 `[ 0, 0, 0, 0, 1, 2, 3, 4, 4, 4 ]` 에 해당하는 행만 읽어오는 효과가 있는 것을 확인 할 수 있다.

### Embedding 레이어 선언

In [3]:
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


 기본적으로 딥러닝은 미분을 기반으로 동작하는데, Embedding 레이어는 그저 단어를 대응 시켜 줄 뿐이니 미분이 불가능하다.  
 따라서 신경망 설계를 할 때 __어떤 연산 결과를 Embedding 레이어에 연결시키는 것은 불가능__ 하다.

__Embedding 레이어는 입력에 직접 연결되게 사용__해야 한다.

----
## RNN

In [4]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


떤 문장이 긍정인지 부정인지 나누기 위해서라면 문장을 모두 읽은 후, 최종 Step의 Output만 확인해도 판단이 가능하다.   


하지만 문장을 생성하는 경우라면 이전 단어를 입력으로 받아 생성된 모든 다음 단어, 즉 모든 Step에 대한 Output이 필요하다. 그것은 위 코드에서 `tf.keras.layers.SimpleRNN` 레이어의 `return_sequences` 인자를 조절함으로써 조절할 수 있다.

![image](https://user-images.githubusercontent.com/84179578/129540191-fb32a84a-821d-4c17-a0f6-de8dd782ec94.png)

- (좌) 모든 Step에 대한 Output이 필요한 경우 (return_sequences=True)  
- (우) 최종 Step에 대한 Output만 필요한 경우 (return_sequences=False)

## LSTM (Long Short-Term Memory)

위 코드에서 RNN 레이어에 대한 부분은 이렇게 바꿀 수도 있다. 동작하는 방식은 위 코드와 동일함

In [5]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


LSTM 레이어를 사용하는 경우, Embedding 벡터의 차원수(unit) 의 크기가 동일 할 때, Weight 의 크기가 위에서 사용했던 SimpleRNN 의 4배나 되는 것을 확인 할 수 있다.

## GRU (Gated Recurrent Unit)

LSTM의 변형 모델로 소개되는 Gated Recurrent Unit(GRU)도 `tf.keras.layers.GRU()` 로 선언해서 사용할 수 있다.

In [6]:
gru_seq_layer = tf.keras.layers.GRU(units=64, return_sequences=True, use_bias=False)
gru_seq_out = gru_seq_layer(emb_out)

print("\nGRU 결과 (모든 Step Output):", gru_seq_out.shape)
print("GRU Layer의 Weight 형태:", gru_seq_layer.weights[0].shape)

gru_fin_layer = tf.keras.layers.GRU(units=64, use_bias=False)
gru_fin_out = gru_fin_layer(emb_out)

print("\nGRU 결과 (최종 Step Output):", gru_fin_out.shape)
print("GRU Layer의 Weight 형태:", gru_fin_layer.weights[0].shape)


GRU 결과 (모든 Step Output): (1, 5, 64)
GRU Layer의 Weight 형태: (100, 192)

GRU 결과 (최종 Step Output): (1, 64)
GRU Layer의 Weight 형태: (100, 192)


LSTM에 비해 GRU가 학습할 가중치(Weight)가 더 적은 것을 확인 할 수 있다.

## 양방향(Bidirectional) RNN

In [9]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (모든 Step Output):", bi_out.shape)



bi_fin = tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False,))

bi_fin_out = bi_fin(emb_out)

print("\nBidirectional RNN 결과 (최종 Step Output):", bi_fin_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (모든 Step Output): (1, 5, 128)

Bidirectional RNN 결과 (최종 Step Output): (1, 128)


Bidirectional RNN은 순방향 Weight와 역방향 Weight를 각각 정의하므로 위에서 다룬 RNN의 2배 크기 Weight가 정의된다.